・testデータも同じようにファイルをつくるべき？
・そうなると特徴量管理はどうなる？

# 参考

In [ ]:
# https://zenn.dev/mst8823/articles/da505dcf45474f

# Pipeline

## Library

In [ ]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import pytz
import sys
import re

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, mean_squared_error

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras import backend as K
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.utils import plot_model

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import pickle
import glob

# import shap
import xgboost
from scipy.stats import spearmanr
from sklearn.ensemble import (
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

warnings.filterwarnings("ignore")

In [ ]:
NAN_VALUE = -127

In [ ]:
def read_file(path = '', usecols = None):
    # LOAD DATAFRAME
    if usecols is not None: df = pd.read_parquet(path, columns=usecols)
    else: df = pd.read_parquet(path)
    # REDUCE DTYPE FOR CUSTOMER AND DATE
    # df['customer_ID'] = df['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    df.S_2 = pd.to_datetime(df.S_2)
    # SORT BY CUSTOMER AND DATE (so agg('last') works correctly)
    #df = df.sort_values(['customer_ID','S_2'])
    #df = df.reset_index(drop=True)
    # FILL NAN
    df = df.fillna(NAN_VALUE) 
    print('shape of data:', df.shape)
    
    return df

TRAIN_PATH = '/content/drive/MyDrive/kaggle/AmericanExpress/INPUT/train.parquet'
TEST_PATH = '/content/drive/MyDrive/kaggle/AmericanExpress/INPUT/test.parquet'
train = read_file(path = TRAIN_PATH)
test = read_file(path = TEST_PATH)

shape of data: (5531451, 190)
shape of data: (11363762, 190)


In [ ]:
train.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
count,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,...,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06,5.531451e+06
mean,-4.045283e-01,5.035986e+00,1.225995e-01,5.653989e-01,7.632776e-02,-2.357494e+01,9.127810e-03,8.465820e-02,-1.096642e+02,-3.834475e+01,...,-9.313243e-01,-9.644764e-01,-9.535646e-01,1.523781e-01,1.413933e-02,-2.164804e+00,-1.061202e+02,1.522530e-01,-8.877382e-01,6.007445e-01
std,1.154356e+01,9.181833e+00,2.131456e-01,2.442297e+00,2.221323e-01,5.021107e+01,2.431505e+00,2.431567e+00,4.456914e+01,5.765541e+01,...,3.926067e-01,1.868469e-01,2.629447e-01,4.072783e-01,1.692060e-01,1.693705e+01,4.685270e+01,4.071715e-01,1.082522e+01,2.119894e+00
min,-1.270000e+02,0.000000e+00,-7.588799e+00,-1.270000e+02,1.534223e-09,-1.270000e+02,-1.270000e+02,-1.270000e+02,-1.270000e+02,-1.270000e+02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.270000e+02,-1.270000e+02,-1.000000e+00,-1.270000e+02,-1.000000e+00
25%,4.745628e-01,0.000000e+00,8.863645e-03,1.050166e-01,2.895934e-03,7.924471e-02,0.000000e+00,5.222354e-03,-1.270000e+02,-1.270000e+02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.270000e+02,0.000000e+00,2.691620e-03,0.000000e+00
50%,6.911091e-01,0.000000e+00,3.132968e-02,8.143263e-01,5.782230e-03,1.495747e-01,0.000000e+00,9.773872e-03,-1.270000e+02,4.683995e-02,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.270000e+02,0.000000e+00,5.467153e-03,0.000000e+00
75%,8.636537e-01,8.000000e+00,1.259019e-01,1.002400e+00,8.660590e-03,2.131741e-01,0.000000e+00,1.549265e-01,-1.270000e+02,1.323237e-01,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.270000e+02,0.000000e+00,8.240029e-03,0.000000e+00
max,1.010000e+00,1.830000e+02,1.324060e+00,1.010000e+00,3.256284e+00,5.482888e+00,8.988807e+00,1.625262e+00,4.191119e+00,1.011162e+01,...,7.000000e+00,1.000000e+00,6.000000e+00,1.000000e+00,1.000000e+00,1.339910e+00,2.229368e+00,1.000000e+00,1.343331e+00,5.300000e+01


In [ ]:
def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM 
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
    num_features = [col for col in all_cols if col not in cat_features]

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape)
    
    return df

In [ ]:
train = process_and_feature_engineer(train)

shape after engineering (458913, 918)


In [ ]:
test_1, test_2 = train_test_split(test, test_size=0.5, shuffle=False)
test_1, test_2, test_3, test_4 = train_test_split(test_1, test_2, test_size=0.5, shuffle=False)
print(len(test_1), len(test_2))

2840940 2840941


In [ ]:
test_1 = process_and_feature_engineer(test_1)
test_2 = process_and_feature_engineer(test_2)
test_3 = process_and_feature_engineer(test_3)
test_4 = process_and_feature_engineer(test_4)

shape after engineering (231134, 918)
shape after engineering (231246, 918)
shape after engineering (231059, 918)
shape after engineering (231185, 918)


In [ ]:
test = pd.concat([test_1, test_2, test_3, test_4])

In [ ]:
test.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.601387,0.020190,0.568930,0.631315,0.568930,2.222222,3.527668,0,8,4,...,1,9,3,2,9,-1,1,9,6,2
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,4,...,1,13,0,1,13,-1,1,13,6,1
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.748955,0.061456,0.673112,0.835114,0.697522,6.000000,9.000000,0,23,0,...,1,13,3,2,13,1,1,13,4,2
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.474728,0.028856,0.428457,0.514222,0.513186,15.846154,4.355957,7,23,11,...,1,13,2,1,13,-1,1,13,5,1
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,26,...,1,13,2,2,13,-1,1,13,5,2


In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/kaggle/AmericanExpress/INPUT/train_labels.csv')
# targets['customer_ID'] = targets['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
targets = targets.set_index('customer_ID')
train = train.merge(targets, left_index=True, right_index=True, how='left')
del targets

train = train.reset_index()
test = test.reset_index()

TRAIN_FEATURES = train.columns[1:-1]
TEST_FEATURES = test.columns[1:-1]

print(f'There are {len(TRAIN_FEATURES)} train_features & {len(TEST_FEATURES)} test_features!')

There are 918 train_features & 917 test_features!


# 新增區段

In [ ]:
train.to_parquet('/content/drive/MyDrive/kaggle/AmericanExpress/INPUT/train_agg.parquet')
test.to_parquet('/content/drive/MyDrive/kaggle/AmericanExpress/INPUT/test_agg.parquet')